# Starsim learning day - Interventions and Analyzers

This notebook illustrates some simple examples of how to construct and use Interventions and Analyzers. It is intended to be read in conjunction with the learning day presentation slides. 

Interventions are used to apply dynamic changes to a `Sim` during execution. This makes them useful for modelling the impact of interventions, including products such as vaccines and treatments, or NPIs that change transmission risks. Analyzers are used to extract values and perform post-processing during execution, which gives them full access to the states of the agents at each timestep.

Both of these classes can be used to extend and add functionality to existing simulations. To demonstrate the usage of these two classes, we begin by setting up a basic SIR model (using the SIR disease module that comes with with Starsim) with a dynamic random network, where each agent is randomly assigned a number of contacts at each timestep. 

In [ ]:
import starsim as ss
import numpy as np
import matplotlib.pyplot as plt

disease = ss.SIR(dur_inf=10, beta=0.1) # SIR model
network = ss.RandomNet(n_contacts=4) # Random network

# Interventions

## Vaccine intervention example

This example shows how to use built-in products and interventions for common use cases like vaccines.

In [2]:
# Create a vaccine product with 50% efficacy
my_vaccine_product = ss.sir_vaccine(efficacy=0.5)

# Create an intervention to distribute the product
vaccine_campaign = ss.routine_vx(
    start_year = 2015,     # Begin vaccination in 2015
    prob = 0.2,            # 20% coverage
    product = my_vaccine_product   # Use the vaccine product created above
)

In [ ]:
# Now create two sims: a baseline sim and one with the intervention
sim_base = ss.Sim(diseases=disease, networks=network, label='Baseline')
sim_base.run();
sim_vac = ss.Sim(diseases=disease, networks=network, interventions=vaccine_campaign, label='Vaccines')
sim_vac.run();

In [ ]:
def plot_sims(*sims, quantity='prevalence'):
    plt.figure()
    for sim in sims:
        plt.plot(sim.results.timevec, sim.results.sir[quantity], label=sim.label)
    plt.axvline(x=2015, color='k', ls='--')
    plt.title(quantity.title())
    plt.legend()
    
plot_sims(sim_base, sim_vac)

## NPI intervention example

In [5]:
class Masks(ss.Intervention):
    def step(self):
        sim = self.sim
        if sim.now > 2015:
            sim.diseases.sir.pars.beta = 0.01 # 10-fold reduction in beta

In [ ]:
sim_masks = ss.Sim(diseases=disease, networks=network, interventions=Masks(), label='Masks')
sim_masks.run()

plot_sims(sim_base, sim_vac, sim_masks)

## Network intervention

In [7]:
class DecreaseContacts(ss.Intervention):
    def step(self):
        sim = self.sim
        if sim.now > 2015:
            sim.networks.randomnet.pars.n_contacts = 1 # Reduce the number of contacts

In [ ]:
# Now run and plot the simulation with the intervention
sim_contacts = ss.Sim(diseases=disease, networks=network, interventions=DecreaseContacts(), label='Fewer contacts')
sim_contacts.run()

plot_sims(sim_base, sim_vac, sim_masks, sim_contacts)

## Apply all interventions

In [ ]:
# Now run and plot the simulation with the intervention
sim_all = ss.Sim(diseases=disease, networks=network, interventions=[vaccine_campaign, Masks(), DecreaseContacts()], label='All interventions')
sim_all.run()

plot_sims(sim_base, sim_vac, sim_masks, sim_contacts, sim_all)

# Analyzers

## Simple analyzer integrated with simulation outputs

In [10]:
# Define a class to count child infections

class ChildInfections(ss.Analyzer):

    def init_pre(self, sim):
        super().init_pre(sim)
        self.define_results(ss.Result('child_infections'))

    def step(self):
        sim = self.sim
        self.results['child_infections'][sim.ti] = sim.diseases.sir.infected[sim.people.age<=15].sum()

In [ ]:
# Now run and plot the simulation with the intervention
# Analyzer.results are automatically added to the simulation results for exporting etc.

sim = ss.Sim(diseases=disease, networks=network, analyzers=ChildInfections())
sim.run()
df = sim.to_df()
df[['sir_n_infected','childinfections_child_infections']].plot();

## Custom analyzer for unique quantities

In [12]:
# Define a class to track the age distribution of infections over time
# This cannot be stored as part of the simulation results as the dimensionality is different
# Instead, the analyzer can store the output directly, and provide methods to render/export it

class AgeHistogram(ss.Analyzer):
    
    def init_pre(self, sim):
        super().init_pre(sim)
        self.ages = np.arange(0,100)
        self.timevec = sim.timevec
        self.infections = np.zeros((len(self.ages)-1, sim.npts))
        
    def step(self):
        sim = self.sim
        self.infections[:,sim.ti] = np.histogram(sim.people.age[sim.diseases.sir.infected], self.ages)[0]
        
    def plot(self):
        plt.figure()
        plt.imshow(analyzer.infections, extent=[self.timevec[0], self.timevec[-1], analyzer.ages.min(), analyzer.ages.max()], origin='lower', aspect='auto', cmap='viridis')
        plt.xlabel('Years')
        plt.ylabel('Age')
        cbar = plt.colorbar(label='Number of infections')
        plt.title('Number of infections by age')

In [ ]:
sim = ss.Sim(diseases=disease, networks=network, analyzers=AgeHistogram())
sim.run();

In [ ]:
analyzer = sim.analyzers[0]
analyzer.plot()